# Summarize large text

<p style="font-size: 20px; font-weight: bold; font-style: italic;">...via LLM-graph</p>

Anton Antonov   
August, September 2025

----

## Introduction

This notebook illustrates how to specify a Large Language Model (LLM) graph for deriving comprehensive summaries of large texts. 
The LLM graph is based on different LLM- and non-LLM functions .
The Raku package ["LLM::Graph"](https://raku.land/zef:antononcube/LLM::Graph) is used, [AAp1].

Using the LLM graph is an alternative to the Literate programming based solutions shown in [AA1, AAn1].

----

## Setup

In [1]:
use LLM::Graph;
use Data::Importers;

use LLM::Tooling;

In [2]:
sink my $conf5-mini = llm-configuration('ChatGPT', model => 'gpt-5-mini');
sink my $conf5 = llm-configuration('ChatGPT', model => 'gpt-5');
sink my $conf41-mini = llm-configuration('ChatGPT', model => 'gpt-4.1-mini', temperature => 0.55, max-tokens => 4096);
sink my $conf41 = llm-configuration('ChatGPT', model => 'gpt-4.1', temperature => 0.45, max-tokens => 8192);
sink my $conf4o-mini = llm-configuration('ChatGPT', model => 'gpt-4o-mini', temperature => 0.45, max-tokens => 8192);
sink my $conf4o = llm-configuration('ChatGPT', model => 'gpt-4o', temperature => 0.45, max-tokens => 8192);

----

## LLM graph

Specify the LLM graph nodes:

In [3]:
sink my %rules =
TypeOfInput => sub ($_) {
        "Determine the input type of\n\n$_.\n\nThe result should be one of: 'Text', 'URL', 'FilePath', or 'Other'."  ~ 
        llm-prompt('NothingElse')('single string')
    },

IngestText =>  { eval-function => sub ($TypeOfInput, $_) { $TypeOfInput ~~ / URL | FilePath/ ?? data-import($_) !! $_} },

Title => { 
    eval-function => sub ($IngestText, $with-title = Whatever) { $with-title ~~ Str:D ?? $with-title !! llm-synthesize([llm-prompt("TitleSuggest")($IngestText, 'article'), "Short title with less that 6 words"]) },
},

Summary => sub ($IngestText) { llm-prompt("Summarize")() ~ "\n\n$IngestText" },

TopicsTable => sub ($IngestText) { llm-prompt("ThemeTableJSON")($IngestText, 'article', 20) },

ThinkingHats => sub ($IngestText) { llm-prompt("ThinkingHatsFeedback")($IngestText, <yellow grey>, format => 'HTML') },

MindMap => sub ($IngestText) { llm-prompt('MermaidDiagram')($IngestText) },

Report => { eval-function => 
    sub ($Title, $Summary, $TopicsTable, $MindMap, $ThinkingHats) { 
        [
            "# $Title",
            '### *LLM summary report*',
            '## Summary',
            $Summary,
            '## Topics',
            to-html(
                from-json($TopicsTable.subst(/ ^ '```json' | '```' $/):g),
                field-names => <theme content>,
                align => 'left'),
            "## Mind map",
            $MindMap,
            '## Thinking hats',
            $ThinkingHats.subst(/ ^ '```html' | '```' $/):g
        ].join("\n\n")
    } 
},

ExportAndOpen => {
    eval-function => sub ($Report) {
       spurt('./Report.md', $Report);
       shell "open ./Report.md" 
    },
    test-function => -> $export-and-open = True { $export-and-open ~~ Bool:D && $export-and-open || $export-and-open.Str.lc ∈ <true yes open> }
}
;

Make the graph:

In [4]:
my $gCombinedSummary = llm-graph(%rules, llm-evaluator => $conf41-mini, :async, :progress)

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

----

## Full computation

URL and text statistics:

In [5]:
my $url = 'https://raw.githubusercontent.com/antononcube/RakuForPrediction-blog/refs/heads/main/Data/Graph-neat-examples-in-Raku-Set-2-YouTube.txt';
my $txtFocus = data-import($url);

text-stats($txtFocus)

(chars => 5957 words => 1132 lines => 157)

Computation:

In [ ]:
$gCombinedSummary.eval({'$_' => $url, with-title => '«Graph» neat examples, set 3'})

**Remark:** Instead of deriving the title using an LLM, the title is specified as an argument.

Show the corresponding graph:

In [ ]:
#% html
$gCombinedSummary.dot(node-width => 1.2, theme => 'ortho'):svg

Final result:

In [ ]:
#% markdown
$gCombinedSummary.nodes<Report><result>.subst(/'```html' | '```' $/):g

---

## Partial evaluation

Drop the results in `LLM::Graph` computed above:

In [6]:
$gCombinedSummary.clear

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

Here the are normalized nodes without results:

In [7]:
.say for |$gCombinedSummary.nodes

Summary => sub { }
Title => {eval-function => sub { }}
ThinkingHats => sub { }
IngestText => {eval-function => sub { }}
MindMap => sub { }
TopicsTable => sub { }
Report => {eval-function => sub { }}
TypeOfInput => sub { }
ExportAndOpen => {eval-function => sub { }, test-function => -> $export-and-open = Bool::True { #`(Block|6072790532248) ... }}


Here all results are pre-assigned as arguments:

In [10]:
$gCombinedSummary.eval({
    '$_' => $url, 
    Title => '«Graph» neat examples, set 3', 
    :export-and-open,
    TypeOfInput => 'Other',
    Summary => 'In brief',
    IngestText => 'Ingest text',
    TopicsTable => '["TopicsTable"]',
    ThinkingHats => 'Thinking hats',
    MindMap => 'Mind-map graph'
})

LLM::Graph(size => 9, nodes => ExportAndOpen, IngestText, MindMap, Report, Summary, ThinkingHats, Title, TopicsTable, TypeOfInput)

-----

## References

### Blog posts

[AA1] Anton Antonov,
["Parameterized Literate Programming"](https://rakuforprediction.wordpress.com/2025/06/21/parameterized-literate-programming/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

### Notebooks

[AAn1] Anton Antonov,
["LLM comprehensive summary template for large texts"](https://community.wolfram.com/groups/-/m/t/3448842),
(2025),
[Wolfram Community](https://community.wolfram.com).

### Packages

[AAp1] Anton Antonov, 
[LLM::Graph, Raku package](https://github.com/antononcube/Raku-LLM-Graph),
(2025),
[GitHub/antononcube](https://github.com/antononcube).